In [9]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**IMPORT LIBRARIES**


In [91]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
%matplotlib inline
sns.set_style('whitegrid')

import scipy 
import warnings

from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import ExtraTreesClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

from sklearn.model_selection import train_test_split

warnings.filterwarnings('ignore')


In [92]:
dataframe = pd.read_csv("../input/wine-quality-dataset/WineQT.csv")

**EDA**

In [12]:
#dataset info
dataframe.info()

In [14]:
dataframe.head(10)

In [93]:
df = dataframe[dataframe.columns]
cm = sns.light_palette("blue", as_cmap=True)
df.head(10).style.background_gradient(cmap=cm)

In [29]:
dataframe[dataframe.quality == 3].head(10).style.background_gradient(cmap=sns.light_palette("green", as_cmap=True))

In [33]:
dataframe[dataframe.quality == 4].head(5).style.background_gradient(cmap=sns.light_palette("red", as_cmap=True))

In [35]:
dataframe[dataframe.quality == 6].head(5).style.background_gradient(cmap=sns.light_palette("orange", as_cmap=True))

In [39]:
# null values in our dataset
pd.DataFrame(df.isnull().sum(), columns=["total"])

In [40]:
# number of unique values in each columns
pd.DataFrame(dataframe.nunique().sort_values(), columns=["numbers"])

In [41]:
total = {}
for value in dataframe.quality:
    if value not in total:
        total[value] = 0
        total[value] += 1

In [94]:
features = df.columns
features = features[0:len(df.columns)-2]
print(f"Total features: {len(features)}")

In [95]:
import random

def get_random_color():
    r1 = lambda: random.randint(0,255)
    return '#%02X%02X%02X' % (r1(),r1(),r1())

def get_histplot_central_tendency(df: dict, fields: list):
    for field in fields:
        f, (ax1) = plt.subplots(1, 1, figsize=(9, 4))
        v_dist_1 = df[field].values
        sns.histplot(v_dist_1, ax=ax1, color=get_random_color(), kde=True)
        
        mean = df[field].mean()
        median=df[field].median()
        mode=df[field].mode().values[0]
        
        ax1.axvline(mean, color='r', linestyle='--', label="Mean")
        ax1.axvline(median, color='g', linestyle='--', label="Mean")
        ax1.axvline(mode, color='b', linestyle='--', label="Mode")
        ax1.legend()
        plt.grid()
        plt.title(f"{field} - Histogram analysis")
        
get_histplot_central_tendency(dataframe, features)        

In [96]:
values = [vl for vl in total.values()]
names = [nm for nm in total]

In [98]:
plt.figure(figsize=(7,7))
plt.pie([values][0], radius = 1, shadow=True, labels=[names][0], counterclock=False, autopct = '%1.1f%%', pctdistance=0.9, explode=[0,0,0,0,0,0])
plt.pie([df.shape[0]], labels=['100%'], labeldistance=-0, radius=0.78, shadow=True, colors=['powderblue'])
plt.title("quality proportion in dataset")
plt.show()

In [99]:
def get_heatmap(df: dict):
    corr = df.corr()
    plt.figure(figsize=(35, 35))
    sns.heatmap(corr, annot=True, cmap="YlGnBu", linewidths=0.1, annot_kws={"fontsize":30})
    plt.title("Correlation whine classification")

In [61]:
get_heatmap(dataframe)

In [70]:
ff=dataframe.corr()
plt.figure(figsize=(35, 35))
sns.heatmap(ff, annot=True, cmap="YlGnBu", linewidths=0.2, annot_kws={"fontsize":14})

**Modelling**

In [72]:
print(f'checking null values of the given dataset: \n{df.isnull().sum()}')

In [100]:
# defining the target values, train, test data set

X = dataframe[features]
y = dataframe["quality"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)
print(f"training {X_train.shape}, Test{X_test.shape}")

In [101]:
# standardizig the dataset

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

print(' Train dataset after scaler')
pd.DataFrame(X_train).head(5).style.background_gradient(cmap=sns.light_palette("green", as_cmap=True))


In [102]:
print(' Test dataset after scaler')
pd.DataFrame(X_test).head(5).style.background_gradient(cmap=sns.light_palette("green", as_cmap=True))

In [103]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

parameters = {'n_estimors': [100, 200, 250], 'max_depth':[14,15,18]}

model_xgb = XGBClassifier(learning_rate=0.3, n_estimator=300, max_depth=14, verborsity=0)
clf = GridSearchCV(model_xgb, parameters)
print("tarining......")
clf.fit(X_train, y_train)
print("training finished")

In [104]:
clf.cv_results_

In [105]:
print(f'Best Score: {clf.best_score_}')
print(f'Best Hyperparameters: {clf.best_params_}')
model_xgb = XGBClassifier(**clf.best_params_)
model_xgb.fit(X_train, y_train)

In [106]:
y_pred = model_xgb.predict(X_test)
accuracy_score(y_test, y_pred)